In [4]:
# Imports

import sys
import time
import string
import json
sys.path.append('../../')
# General
import numpy as np
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from contextlib import redirect_stdout
# brain2brain
from brain2brain import utils
from brain2brain import generators
# TCN
from brain2brain.tcn import TCN
from brain2brain.tcn import compiled_tcn
# TF

from tensorflow.keras import backend as K, Model, Input, optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Flatten, GRU, Lambda, TimeDistributed, Activation
# import wandb


In [ ]:
%cd ..

In [6]:
target_folder = experiment_dict['target_folder']
path = experiment_dict['path']
batch_size = experiment_dict['batch_size']
epochs = experiment_dict['epochs']
lookback_window = experiment_dict['lookback_window']
length_pred = experiment_dict['length_pred']
delay_pred = experiment_dict['delay_pred']
samples_per_second = experiment_dict['samples_per_second']
electrode_selection=experiment_dict['electrode_selection']
debug_mode = experiment_dict['debug_mode']
num_feat = experiment_dict['num_feat']
num_classes = experiment_dict['num_classes']
nb_filters = experiment_dict['nb_filters']
kernel_size = experiment_dict['kernel_size']
dilations=experiment_dict['dilations']
nb_stacks = experiment_dict['nb_stacks']
output_len = experiment_dict['output_len']
padding = experiment_dict['padding']
use_skip_connections = experiment_dict['use_skip_connections']
return_sequences = experiment_dict['return_sequences']
regression = experiment_dict['regression']
dropout_rate = experiment_dict['dropout_rate']
name = experiment_dict['name']
kernel_initializer = experiment_dict['kernel_initializer']
activation = experiment_dict['activation']
opt = experiment_dict['opt']
lr = experiment_dict['lr']

# Read saved paths for training.
saved_paths = utils.get_file_paths(path)
# Split the train files into a training and validation set.
train, val = utils.split_file_paths(saved_paths, 0.8)
total_electrode_count = utils.get_file_shape(train[0])[1]
# Electrodes
electrode_count = len(electrode_selection)
# Sampling of electrodes.
timesteps_per_sample = int(lookback_window // samples_per_second)
# Training Generator

(36853, 114)


In [7]:
# Training Generator
train_generator = generators.FGenerator(file_paths=train,
                                        lookback=lookback_window, length=length_pred, delay=delay_pred,
                                        batch_size=batch_size, sample_period=samples_per_second,
                                        electrodes=electrode_selection, shuffle=True, debug=debug_mode)
# Validation Generator
val_generator = generators.FGenerator(file_paths=val,
                                      lookback=lookback_window, length=length_pred, delay=delay_pred,
                                      batch_size=batch_size, sample_period=samples_per_second,
                                      electrodes=electrode_selection, shuffle=False, debug=debug_mode)

In [8]:
x, y = train_generator[0]
x.shape

(1024, 128, 5)

In [9]:
x[0, :3, 1]

array([0.22746221, 0.34503541, 0.43942214])

In [ ]:
# GRU
model = Sequential()
model.add(GRU(units=timesteps_per_sample,
              dropout=0,
              recurrent_dropout=0,
              input_shape=(timesteps_per_sample, electrode_count),
              return_sequences = return_sequences))
# model.add(Dense(electrode_count))
model.add(TimeDistributed(Dense(electrode_count)))
# model.add(Activation("linear"))
model.add(Lambda(lambda x: x[:, :length_pred , :]))

model.compile(optimizer=RMSprop(), loss="mae")

# Save Summary
model.summary()

In [21]:
with open("../experiment_params/tcn_experiment_m2m_4.json") as f:
    experiment_dict = json.load(f)

In [22]:
target_folder = experiment_dict['target_folder']
path = experiment_dict['path']
batch_size = experiment_dict['batch_size']
epochs = experiment_dict['epochs']
lookback_window = experiment_dict['lookback_window']
length_pred = experiment_dict['length_pred']
delay_pred = experiment_dict['delay_pred']
samples_per_second = experiment_dict['samples_per_second']
electrode_selection=experiment_dict['electrode_selection']
debug_mode = experiment_dict['debug_mode']
num_feat = experiment_dict['num_feat']
num_classes = experiment_dict['num_classes']
nb_filters = experiment_dict['nb_filters']
kernel_size = experiment_dict['kernel_size']
dilations=experiment_dict['dilations']
nb_stacks = experiment_dict['nb_stacks']
output_len = experiment_dict['output_len']
padding = experiment_dict['padding']
use_skip_connections = experiment_dict['use_skip_connections']
return_sequences = experiment_dict['return_sequences']
regression = experiment_dict['regression']
dropout_rate = experiment_dict['dropout_rate']
name = experiment_dict['name']
kernel_initializer = experiment_dict['kernel_initializer']
activation = experiment_dict['activation']
opt = experiment_dict['opt']
lr = experiment_dict['lr']
# Read saved paths for training.
saved_paths = utils.get_file_paths(path)
# Split the train files into a training and validation set.
train, val = utils.split_file_paths(saved_paths, 0.8)
total_electrode_count = utils.get_file_shape(train[0])[1]
# Electrodes
electrode_count = len(electrode_selection)
# Sampling of electrodes.
timesteps_per_sample = int(lookback_window // samples_per_second)
# Training Generator

(36853, 114)


In [24]:
# Training Generator
train_generator = generators.FGenerator(file_paths=train,
                                        lookback=lookback_window, length=length_pred, delay=delay_pred,
                                        batch_size=batch_size, sample_period=samples_per_second,
                                        electrodes=electrode_selection, shuffle=True, debug=debug_mode)
# Validation Generator
val_generator = generators.FGenerator(file_paths=val,
                                      lookback=lookback_window, length=length_pred, delay=delay_pred,
                                      batch_size=batch_size, sample_period=samples_per_second,
                                      electrodes=electrode_selection, shuffle=False, debug=debug_mode)

In [23]:
def get_opt(opt):
    if opt == "adam":
        return optimizers.Adam(lr=lr, clipnorm=1.0)
    elif opt == "rmsprop":
        return optimizers.RMSprop(lr=lr, clipnorm=1.0)
    else:
        raise Exception("Only Adam and RMSProp are available here.")


# TCN
input_layer = Input(shape=(lookback_window, electrode_count))
x = TCN(nb_filters=nb_filters,
        kernel_size=kernel_size,
        dilations=dilations,
        nb_stacks=nb_stacks,
        padding=padding,
        use_skip_connections=use_skip_connections,
        return_sequences=return_sequences,
        activation=activation,
        dropout_rate=dropout_rate,
        kernel_initializer=kernel_initializer)(input_layer)
# x = TimeDistributed(Dense(electrode_count))(x)
x = Dense(electrode_count)(x)
# x = Lambda(lambda x: x[:, :length_pred , :])(x)
# x = Activation('linear')(x)
# model.add(Dense(electrode_count))
# model.add(TimeDistributed(Dense(electrode_count)))
# model.add(Activation("linear"))
# model.add(Lambda(lambda x: x[:, :length_pred , :]))
output_layer = x
model = Model(input_layer, output_layer)
model.compile(get_opt("rmsprop"), loss='mean_absolute_error')

print('model.x = {}'.format(input_layer.shape))
print('model.y = {}'.format(output_layer.shape))

# Save Summary
model.summary()

(None, 128, 5)
model.x = (None, 128, 5)
model.y = (None, 5)
Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 128, 5)]          0         
_________________________________________________________________
tcn_6 (TCN)                  (None, 16)                21904     
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 85        
Total params: 21,989
Trainable params: 21,989
Non-trainable params: 0
_________________________________________________________________


In [25]:
val_generator[0][1][0].shape

(1, 5)